In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

In [ ]:
#Install Libraries
!pip install -q twilio pydub
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install faster-whisper

In [ ]:
# Import Dependencies
import json
import time
import requests
from twilio.rest import Client
from pydub import AudioSegment
from faster_whisper import WhisperModel

In [ ]:
# Twilio Setup -  for calling and recording you can get all these info when you create an account on twilio
import os
TWILIO_ACCOUNT_SID = input("Enter your Twilio Account SID: ") # Your Twilio SID
TWILIO_AUTH_TOKEN = input("Enter your Twilio Auth Token: ")    # Your Twilio Auth Token
TWILIO_PHONE_NUMBER = input("Enter your Twilio Phone Number: ")   # Your Twilio Number (e.g., +1415xxxxxx)
TWIML_BIN_URL = input("Enter your TwiML Bin URL: ") #url created in twiml bin in xml for twilio calling tone and audio like what to speak
OPENROUTER_API_KEY = input("Enter your OpenRouter API Key: ")
MODEL = input("Enter Your Model Name: ")

client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

In [ ]:
# this will ask for candidate's mobile number then Triggers the Call
CANDIDATE_PHONE = input("Enter candidate number (e.g., +91xxxxxxxxxx): ")
call = client.calls.create(
    to=CANDIDATE_PHONE,
    from_=TWILIO_PHONE_NUMBER,
    url=TWIML_BIN_URL
)

print("Call initiated. SID:", call.sid)

In [ ]:
print("Waiting 20 seconds for call and recordings to complete...")
time.sleep(20)

# Fetch recordings
recordings = client.recordings.list(call_sid=call.sid)
print(f"\n Found {len(recordings)} recording(s)")

# Load Whisper model
model = WhisperModel("medium", compute_type="float16")

conversation_log = []

# Process & Transcribe each recording
for idx, rec in enumerate(recordings):
    audio_url = f"https://api.twilio.com{rec.uri.replace('.json', '.wav')}"
    response = requests.get(audio_url, auth=(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN))

    # Save original recording
    raw_file = f"response_{idx+1}.wav"
    with open(raw_file, "wb") as f:
        f.write(response.content)

    # Resample to 16kHz mono
    sound = AudioSegment.from_file(raw_file)
    sound = sound.set_frame_rate(16000).set_channels(1)
    processed_file = f"processed_{idx+1}.wav"
    sound.export(processed_file, format="wav")

    # Transcribe using Whisper
    segments, _ = model.transcribe(processed_file, beam_size=5)
    candidate_response = " ".join([seg.text.strip() for seg in segments])

    conversation_log.append({
        "agent": f"Q{idx+1}",
        "candidate": candidate_response
    })

# Print Final Conversation Log
print("\n Final Transcription Log:\n")
for turn in conversation_log:
    print(f"{turn['agent']}: {turn['candidate']}")

In [ ]:
# LLM Prompt for candidate's review
prompt = f"""
You are a recruitment AI assistant. Here's the transcript of a candidate interview:

---
{conversation_log}
---

Your tasks:
1. Detect sentiment and tone (positive, negative, neutral, confident, hesitant, confused).
2. Identify key information: skills, experience, location, willingness to join.
3. Determine if the candidate asked a question that needs human escalation and note the questions.
4. Recommend a decision: Shortlist, Escalate, or Hold.
5. Suggest a closing response line.

Respond in JSON format with keys:
{{
  "Sentiment": "",
  "Tone": "",
  "Skills": [],
  "Experience": "",
  "Location": "",
  "Willing_to_Join": true,
  "Candidate_Asked_Question": false,
  "Candidate_Asked_Questions_List": [],
  "Decision": "",
  "Closing_Line": ""
}}
"""

In [ ]:
# Call LLM


headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": "You are a helpful recruitment AI assistant."},
        {"role": "user", "content": prompt}
    ]
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)

In [ ]:
import re

try:
    llm_output_raw = response.json()["choices"][0]["message"]["content"]

    # Strip markdown-style formatting like ```json
    llm_output_clean = re.sub(r"```json|```", "", llm_output_raw).strip()

    # Convert to dictionary
    final_output = json.loads(llm_output_clean)

    # Pretty Output (like a report)
    print("\n==============================")
    print(" Candidate Screening Summary")
    print("==============================\n")

    print(f"-> Sentiment       : {final_output.get('Sentiment', 'N/A')}")
    print(f"-> Tone            : {final_output.get('Tone', 'N/A')}")
    print(f"-> Experience      : {final_output.get('Experience', 'N/A')}")
    print(f"-> Skills          : {', '.join(final_output.get('Skills', []))}")
    print(f"-> Location        : {final_output.get('Location', 'N/A')}")
    print(f"-> Willing to Join : {'Yes' if final_output.get('Willing_to_Join') else 'No'}")
    print(f"-> Asked Questions : {'Yes' if final_output.get('Candidate_Asked_Question') else 'No'}")

    if final_output.get("Candidate_Asked_Questions_List"):
        print(" Questions from Candidate:")
        for q in final_output["Candidate_Asked_Questions_List"]:
            print(f"   - {q}")

    print(f"\n Final Decision  : {final_output.get('Decision', 'N/A')}")
    print(f" Closing Line    : {final_output.get('Closing_Line', 'N/A')}")

except Exception as e:
    print("\n LLM Parsing Error:", e)

    if 'response' in locals():
        print("\n Raw response:")
        try:
            print(response.text)
        except:
            print("No .text content in response.")
    else:
        print("No response object was captured.")
